In [1]:
from binance import Client

In [2]:
import pandas as pd

In [3]:
from binance import BinanceSocketManager

In [4]:
import time

In [5]:
import numpy as np

In [6]:
import sys

In [7]:
from decimal import Decimal, ROUND_HALF_UP

In [8]:
%run ./Binance_keys.ipynb

In [9]:
client = Client(api_key, api_secret)

In [10]:
def getminutedata(symbol, interval, lookback):
    frame = pd.DataFrame(client.get_historical_klines(symbol, interval, lookback+' min ago UTC'))
    frame = frame.iloc[:,:6]
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame

In [11]:
def currentprice(symbol):
    dic = client.get_ticker(symbol= 'AVAXUSDT')
    price = dic.get("lastPrice")
    strings = [str(integer) for integer in price]
    a_string = "".join(strings)
    a_string = a_string.split('.', 1)[0]+ '.'+a_string.split('.', 2)[1] 
    a_float = float(a_string)
    return a_float

In [12]:
def getlastsell():
    frame = pd.DataFrame(client.get_my_trades(symbol='AVAXUSDT'))
    frame = frame.iloc[1:, -9:-1]
    frame = frame[(frame['isBuyer']==False)]
    frame = frame.iloc[-1:, :2]
    frame = frame.astype(float)
    frame = frame.set_index('qty')
    #frame = frame.get(key='price')
    dic = frame.to_dict("list")
    listdic = dic.get('price')
    strings = [str(integer) for integer in listdic]
    a_string = "".join(strings)
    a_string = a_string.split('.', 1)[0]+ '.'+a_string.split('.', 2)[1]
    a_float = float(a_string)
    return a_float

In [13]:
def getlastbuy():
    frame = pd.DataFrame(client.get_my_trades(symbol='AVAXUSDT'))
    frame = frame.iloc[1:, -9:-1]
    frame = frame[(frame['isBuyer']==True)]
    frame = frame.iloc[-1:, :2]
    frame = frame.astype(float)
    #frame = frame.get(key='price')
    frame = frame.set_index('qty')
    dic = frame.to_dict("list")
    listdic = dic.get('price')
    strings = [str(integer) for integer in listdic]
    a_string = "".join(strings)
    a_string = a_string.split('.', 1)[0]+ '.'+a_string.split('.', 2)[1]
    a_float = float(a_string)
    return a_float

In [14]:
def getlastbuyqty():
    frame = pd.DataFrame(client.get_my_trades(symbol='AVAXUSDT'))
    frame = frame.iloc[1:, -9:-1]
    frame = frame[(frame['isBuyer']==True)]
    frame = frame.iloc[-1:, :2]
    frame = frame.astype(float)
    #frame = frame.get(key='price')
    frame = frame.set_index('price')
    dic = frame.to_dict("list")
    listdic = dic.get('qty')
    strings = [str(integer) for integer in listdic]
    a_string = "".join(strings)
    a_string = a_string.split('.', 1)[0]+ '.'+a_string.split('.', 2)[1]
    a_float = float(a_string)
    return a_float

In [15]:
def to_money(amount) -> Decimal:
    money = Decimal(amount).quantize(Decimal('.00'), rounding=ROUND_HALF_UP)
    return money

In [16]:
def strategyCrypto(symbol, bought = False):
    while True:
        try:
        
            if not bought:
                while True:
                    framing = getminutedata(symbol, '1m', '10m')
                    actualprice = currentprice(symbol)
                    #numericprice = actualprice.apply(pd.to_numeric)
                    lastsell = getlastsell()
                    #numericsell = lastsell.apply(pd.to_numeric)
                    cumulret = (framing.Open.pct_change() +1).cumprod() -1
                    usdt = client.get_asset_balance(asset='USDT')
                    usdtstr = usdt.get("free")
                    usdtfloat = float(usdtstr)
                    usdtavail = usdtfloat * 1
                    numbercrypto = usdtavail // actualprice
                    #   cumulret[-1] < -0.001 and 
                    if cumulret[-1] < -0.00539:
                    #if (actualprice*1.001) < (lastsell*.997) or cumulret[-1] > 0.023:
                        order = client.create_order(symbol = symbol, side='BUY', type= 'MARKET', quantity = numbercrypto)
                        print(order)
                        bought = True
                        break
                    else:
                        print("No Trade has been executed")
            
            if bought:
                while True:
                    framing = getminutedata(symbol, '1m', '30m')
                    framingUrgent = getminutedata(symbol, '1m', '480m')
                    framingnormal = getminutedata(symbol, '1m', '10m')
                    urgentret = (framingUrgent.Open.pct_change() +1).cumprod() -1
                    normalret = (framingnormal.Open.pct_change() +1).cumprod() -1
                    actualprice = currentprice(symbol)
                    avax = client.get_asset_balance(asset='AVAX')
                    avaxstr = avax.get("free")
                    avaxfloat = float(avaxstr)
                    avaxavail = avaxfloat * 0.47
                    avaxurgent = avaxfloat * 0.955
                    avaxavail = to_money(avaxavail)
                    avaxurgent = to_money(avaxurgent)
                    lastbuy = getlastbuy()
                    lastbuyqty = getlastbuyqty()
                    lastbuyqty = to_money(lastbuyqty)
                    sincebuy = framing.loc[framing.index > pd.to_datetime(order['transactTime'], unit = 'ms')]
                    
                    
                    lastbuyqtyfinal = order.get('executedQty')
                    stringsqty = [str(integer) for integer in lastbuyqtyfinal]
                    qty_string = "".join(stringsqty)
                    qty_string = qty_string.split('.', 1)[0]+ '.'+qty_string.split('.', 2)[1]
                    qty_float = float(qty_string)
                    lastbuyqtyfinalavail = to_money(qty_float)
                    
                    if len(sincebuy) > 0:
                        sincebuyret = (sincebuy.Open.pct_change() +1).cumprod() -1
                        #sincebuyret[-1] > 0.0009 and 
                        if (((actualprice*.997) > (lastbuy*1.001)) and urgentret[-1] > .15):
                            order = client.create_order(symbol = symbol, side='SELL', type= 'MARKET', quantity = avaxurgent)
                            print(order)
                            bought = False
                            time.sleep(5400)
                            break
                        #elif (actualprice*.997) > (lastbuy*1.001):
                        elif (((actualprice*.997) > (lastbuy*1.001)) and normalret[-1] > .00539):
                            order = client.create_order(symbol = symbol, side='SELL', type= 'MARKET', quantity = lastbuyqtyfinalavail)
                            print(order)
                            bought = False
                            break
        except: # catch *all* exceptions
            e = sys.exc_info()[0]
            print("Oops!  That was invalid data.  Try again...")
            print(e)

In [ ]:
strategyCrypto('AVAXUSDT')

No Trade has been executed
No Trade has been executed
No Trade has been executed
No Trade has been executed
No Trade has been executed
No Trade has been executed
Oops!  That was invalid data.  Try again...
<class 'KeyboardInterrupt'>
No Trade has been executed
No Trade has been executed
No Trade has been executed
No Trade has been executed
No Trade has been executed
No Trade has been executed
No Trade has been executed
